# Dialogue Summarization with transformer-based pre-trained models

This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Splunk Machine Learning Toolkit (MLTK) Container for TensorFlow. This script contains an example of how to conduct **End-to-End automatic summarization task over English dialogue transcripts** using the transformers library.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [5]:
# this definition exposes all python module imports that should be available in all subsequent commands
# import sys
# sys.path.insert(1, '/opt/conda/lib/python3.8/site-packages')

import json
import numpy as np
import pandas as pd
import transformers
from transformers import T5ForConditionalGeneration, T5Tokenizer

import os
# path = os.path.abspath(np.__file__)
# print("numpy path in runtime: " + path)
# print("numpy version in runtime: " + np.__version__)

# tokenizer = AutoTokenizer.from_pretrained("/opt/conda/lib/python3.8/site-packages/samsum_tok", local_files_only=True)
# summarizer = AutoModelForSeq2SeqLM.from_pretrained("/opt/conda/lib/python3.8/site-packages/samsum", local_files_only=True)

# tokenizer = AutoTokenizer.from_pretrained("philschmid/bart-large-cnn-samsum")
# # tokenizer.save_pretrained("/opt/conda/lib/python3.8/site-packages/samsum_tok")
# model = AutoModelForSeq2SeqLM.from_pretrained("philschmid/bart-large-cnn-samsum")
# model.save_pretrained("/opt/conda/lib/python3.8/site-packages/samsum")

MODEL_DIRECTORY = "/srv/app/model/data/"

In [6]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.22.1
pandas version: 1.4.3


In [4]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [3]:
# from transformers import pipeline
# summarizer = pipeline("summarization", model="lidiya/bart-base-samsum")

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| makeresults
| eval text = "お客様：こんにちは、；従業者：こんにちは、；お客様、ご機嫌いかがでしょうか？；お客様：こんにちは、元気です。私はwifiサービスの性能に大きな問題を抱えています。先月から御社のwifiサービスを使い始めたのですが、購入したサービスのようにうまく機能していないのです。；従業者：今、ユーザー番号を確認します。お客様のユーザー番号は100145で、先月に作成されたものですが、よろしいですか？；お客様：はい ；従業者：お客様さん、何が問題なのか説明してもらえますか？；お客様：はい。私はあなたの無線LANのプレミアムバージョンに加入しており、それは通常の加入よりも高速であるはずです。しかし、とても遅いのです。；従業者：しかし、私たちのセンターから提供する無線LANサービスは、あなたが契約したパッケージの説明とまったく同じです。；お客様：私は自分の目で見た無線LANの速度を信じています。；従業者：申し訳ございませんが、お客様、私はあなたが嘘をついていると非難するつもりは全くありません、私はすぐにこの問題を解決します、あなたは私に詳細に説明することができますか？無線LANがあなたの期待に比べてどのように遅いですか？；お客様様：無線LANは時々オフラインで、物をダウンロードするのに時間がかかります。；従業者：ローカルネットワークに何らかの問題があるかもしれませんね。ルーターをチェックしていただけますか？；お客様: はい。 ；従業者： ごゆっくり、お客様。；お客様：ルーターの電源は入っていますが、ランプが点滅しています ；従業者：ランプが点滅するのは正常ではありません。ランプは接続の安定性を示すものですから、ランプが点滅するのは正常なことではありません。どのランプが点滅しているのか、またその点滅の速さを知る必要があるかもしれませんが、ご確認いただけますか？；お客様：はい ；従業者：アクティブという名前の光はありますか？；お客様：はい。それは点滅しているものです。；従業者：それはあなたのネットワークの問題であるべきですあなたはすべてのケーブルがうまく接続されていることを確認することができますか？；お客様：はい ；従業者：無線LANを再接続しようとしたことがありますか？；お客様：はい、私はやったし、私は問題があなたの側にあると思う　；従業者：今私はあなたのローカルネットワークのデータ入力を確認します"
| fit MLTKContainer algo=transformers_dialogsum_jp mode=stage epochs=100 text into app:transformers_dialogjp_model as text

In [7]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [8]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("transformers_dialogjp_model")
print(df)
print(df.shape)
print(str(param))

                                                text
0  お客様：こんにちは、；従業者：こんにちは、；お客様、ご機嫌いかがでしょうか？；お客様：こんに...
(1, 1)
{'options': {'params': {'algo': 'transformers_dialogsum_jp', 'mode': 'stage', 'epochs': '100'}, 'args': ['text'], 'feature_variables': ['text'], 'model_name': 'transformers_dialogjp_model', 'output_name': 'text', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '600', 'max_inputs': '100000', 'max_memory_usage_mb': '4000', 'max_model_size_mb': '30', 'max_score_time': '600', 'use_sampling': 'true'}, 'kfold_cv': None}, 'feature_variables': ['text']}


## Stage 2 - create and initialize a model

In [9]:
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    # Load English parser and text blob (for sentiment analysis)
    print("debug: start model initialization")
    model = {}
#     model["tokenizer"] = AutoTokenizer.from_pretrained("philschmid/bart-large-cnn-samsum")
#     model["summarizer"] = TFAutoModelForSeq2SeqLM.from_pretrained("philschmid/bart-large-cnn-samsum")
    model["tokenizer"] = T5Tokenizer.from_pretrained("/srv/app/model/data/t5_dialog_jp")
    model["summarizer"] = T5ForConditionalGeneration.from_pretrained("/srv/app/model/data/t5_dialog_jp")
    print("debug: model initialized")
    return model

In [10]:
model = init(df,param)

debug: start model initialization
debug: model initialized


## Stage 3 - fit the model

Note that for this algorithm the model is pre-trained and therefore this stage is a placeholder only

In [11]:
# returns a fit info json object
def fit(model,df,param):
    returns = {}
    return returns

## Stage 4 - apply the model

In [21]:
def apply(model,df,param):
    
    print("debug: start apply")
    
    X = df[param['feature_variables']].values.tolist()
    temp_data=list()
    
#     tokenizer = AutoTokenizer.from_pretrained("/srv/app/model/data/samsum_tok")
#     summarizer = AutoModelForSeq2SeqLM.from_pretrained("/srv/app/model/data/samsum")
    print("debug: read input")
    print("numpy version: " + np.__version__)
    for i in range(len(X)):
        print("debug: start tokenizing")
        batch = model["tokenizer"](str(X[i]), max_length=400, truncation=True, return_tensors="pt")
        
        print("debug: finish tok and start summarizing")
        
        outputs = model["summarizer"].generate(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], max_length=400,repetition_penalty=8.0,num_beams=15)
        
        print("debug: finish summarizing and start decoding")
        
        summary = [model["tokenizer"].decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False) for ids in outputs][0]
        
        print("debug: finish decoding")
        
        temp_data.append(summary)
        
    column_names=["summaries"]
    returns=pd.DataFrame(temp_data, columns=column_names)
    print("debug: finished")
        
    return returns

In [22]:
returns = apply(model,df,param)
print(returns["summaries"][0])

debug: start apply
debug: read input
numpy version: 1.22.1
debug: start tokenizing
debug: finish tok and start summarizing
debug: finish summarizing and start decoding
debug: finish decoding
debug: finished
無線のプレミアムバージョンに加入しているユーザー番号は100145。通常加入よりも高速であるはずなのに、とても遅いという。お客様が契約したパッケージの説明とまったく同じだそう


## Stage 5 - save the model

In [14]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # model will not be saved or reloaded as it is pre-built
    return model

## Stage 6 - load the model

In [15]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    # model will not be saved or reloaded as it is pre-built
    return model

## Stage 7 - provide a summary of the model

In [16]:
# return model summary
def summary(model=None):
    returns = {}
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code